In [29]:
from sklearn.model_selection import train_test_split
import numpy as np

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
from sklearn.svm import SVC

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
from sklearn.model_selection import KFold

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#kmer1
fs1Train = ["/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_pos/combined/Kmer1/A.thaliana5289indep_pos_kmer_1.npy",
                   "/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_neg/concated result/concated kmer1/A.thaliana5289_neg_kmr1.npy"]
fs1Test = ["/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_pos_kmer_1.npy", 
                  "/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_neg_kmer_1.npy"]


#enac
fs2Train = ["/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_pos/combined/ENAC/A.thaliana5289indep_pos_ENAC.npy",
                   "/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_neg/concated result/concated enac/A.thaliana5289_neg_ENAC.npy"]
fs2Test = ["/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_pos_ENAC.npy", 
                  "/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_neg_enac.npy"]

In [36]:
kmer1_pos_5289 = fs1Train[0]
kmer1_neg_5289 = fs1Train[1]
kmer1_pos_1000 = fs1Test[0]
kmer1_neg_1000 = fs1Test[1]


enac_pos_5289 = fs2Train[0]
enac_neg_5289 = fs2Train[1]
enac_pos_1000 = fs2Test[0]
enac_neg_1000 = fs2Test[1]




x_train_pos = np.concatenate((np.load(kmer1_pos_5289),np.load(enac_pos_5289)),axis=1) 
x_train_neg = np.concatenate((np.load(kmer1_neg_5289),np.load(enac_neg_5289)),axis=1) 
x_test_pos = np.concatenate((np.load(kmer1_pos_1000),np.load(enac_pos_1000)),axis=1)  
x_test_neg = np.concatenate((np.load(kmer1_neg_1000),np.load(enac_neg_1000)),axis=1)  

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000) 

In [37]:
print(x_train_pos.shape)
print(x_train_neg.shape)
print(x_test_pos.shape)
print(x_test_neg.shape)

print(y_train_pos.shape)
print(y_train_neg.shape)
print(y_test_pos.shape)
print(y_test_neg.shape)

(5289, 152)
(5289, 152)
(1000, 152)
(1000, 152)
(5289,)
(5289,)
(1000,)
(1000,)


In [38]:
seed = 40
np.random.seed(seed)

In [39]:
x_training = np.concatenate((x_train_pos, x_train_neg), axis= 0)
y_training = np.concatenate((y_train_pos, y_train_neg))

In [40]:
x_test = np.concatenate((x_test_pos, x_test_neg))
y_test = np.concatenate((y_test_pos, y_test_neg))

In [41]:
print(x_test.shape)
print(y_test.shape)

(2000, 152)
(2000,)


In [42]:
print(x_training.shape)
print(y_training.shape)

(10578, 152)
(10578,)


In [43]:
kf = KFold(n_splits=5, random_state=seed)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [49]:
j=1000
n_list=[]
n_test_avg =[]
n_train_avg = []
n_val_avg = [] 
while j <= 2000:
  n = j
  sumTrain = sumVal = sumTest =  0
  print(f"SVC: kmer1+enac  |   n = {n}")
  clf = RandomForestClassifier(n_estimators = n, random_state=seed)

  val_accuracy_list = []
  train_accuracy_list = []
  test_accuracy_list = []
  n = 1
  for train_index, val_index in kf.split(x_training):
      print("Fold : ",n)
      X_train, X_val = x_training[train_index], x_training[val_index]
      y_train, y_val = y_training[train_index], y_training[val_index]
      
      clf.fit(X_train, y_train)

      y_val_pred = clf.predict(X_val)
      val_accuracy = accuracy_score(y_val, y_val_pred)
      val_accuracy_list.append(val_accuracy)

      y_train_pred = clf.predict(X_train)
      train_accuracy = accuracy_score(y_train, y_train_pred)
      train_accuracy_list.append(train_accuracy)

      y_test_pred = clf.predict(x_test)
      test_accuracy = accuracy_score(y_test, y_test_pred)
      test_accuracy_list.append(test_accuracy)

      n += 1 

  print("done")
  for i in range(0, len(train_accuracy_list)):
      print("train_accuracy : ", train_accuracy_list[i], "\tVal accuracy : ", val_accuracy_list[i], "\tTest accuracy : ", test_accuracy_list[i])
      sumTrain = sumTrain + train_accuracy_list[i]
      sumVal = sumVal + val_accuracy_list[i]
      sumTest = sumTest + test_accuracy_list[i]
  sumTrain = sumTrain/5.0
  sumVal = sumVal/5.0
  sumTest = sumTest/5.0
  print("------------------------------------------------------------------------------------------------")
  print(f"Train accuracy(avg): {sumTrain} | Val accuracy(avg): {sumVal}  | Test accuracy(avg): {sumTest}")
  print("================================================================================================")
  n_list.append(j)
  n_train_avg.append(sumTrain)
  n_val_avg.append(sumVal)
  n_test_avg.append(sumTest)
  j = j+100

SVC: kmer1+enac  |   n = 1000
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done
train_accuracy :  1.0 	Val accuracy :  0.3700378071833648 	Test accuracy :  0.606
train_accuracy :  1.0 	Val accuracy :  0.5647448015122873 	Test accuracy :  0.672
train_accuracy :  1.0 	Val accuracy :  0.719281663516068 	Test accuracy :  0.7855
train_accuracy :  1.0 	Val accuracy :  0.8851063829787233 	Test accuracy :  0.718
train_accuracy :  1.0 	Val accuracy :  0.4557919621749409 	Test accuracy :  0.559
------------------------------------------------------------------------------------------------
Train accuracy(avg): 1.0 | Val accuracy(avg): 0.598992523473077  | Test accuracy(avg): 0.6681
SVC: kmer1+enac  |   n = 1100
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done
train_accuracy :  1.0 	Val accuracy :  0.3724007561436673 	Test accuracy :  0.608
train_accuracy :  1.0 	Val accuracy :  0.5675803402646503 	Test accuracy :  0.6755
train_accuracy :  1.0 	Val accuracy :  0.7188090737240076 	Test 

In [50]:
i=0  
print(" c      train                   val                    test")
print("===================================================================")
for x in n_list:
  print(f"{x}  -  {n_train_avg[i]} - {n_val_avg[i]} - {n_test_avg[i]}")
  print("--------------------------------------------------------------------")
  i=i+1

 c      train                   val                    test
1000  -  1.0 - 0.598992523473077 - 0.6681
--------------------------------------------------------------------
1100  -  1.0 - 0.5996540151139355 - 0.6687000000000001
--------------------------------------------------------------------
1200  -  1.0 - 0.6009771771530208 - 0.6682
--------------------------------------------------------------------
1300  -  1.0 - 0.6015445530395456 - 0.6678
--------------------------------------------------------------------
1400  -  1.0 - 0.5993704612386992 - 0.6678000000000001
--------------------------------------------------------------------
1500  -  1.0 - 0.5984252816545783 - 0.6680999999999999
--------------------------------------------------------------------
1600  -  1.0 - 0.597479833934405 - 0.6689
--------------------------------------------------------------------
1700  -  1.0 - 0.5973850031505986 - 0.6694
--------------------------------------------------------------------
1800  -  1